In [1]:
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder.appName("ADS project 2")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.executor.memory", "2g")
    .config("spark.driver.memory", "4g")
    .getOrCreate()
)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/12 01:00:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/09/12 01:00:47 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
import pandas as pd
import numpy as np

raw_data = "../data/ABS/"
curated_data = "../data/curated/"

tbl_income = pd.read_excel(raw_data+"annual income by SA2 regions.xlsx")
tbl_population = pd.read_excel(raw_data+"population estimates by SA2 regions.xlsx")


<h2>1. Clean external data</h2>

<h3>1.1 Clean "annual income"</h3>

In [3]:
tbl_income.drop(np.arange(0,8), inplace=True)
tbl_income.drop(np.arange(2303,2306), inplace=True)

tbl_income.drop(tbl_income.columns[[2,3,4,5,7,8,9,10,12,13,14,15,17,18,19,20,22,23,24,25,27]], axis=1, inplace=True)
tbl_income = tbl_income.reset_index(drop=True)
tbl_income.columns = ['sa2', 'sa2_name', 'total_earners', 'median_age', 'income_sum', 'income_median', 'income_mean']
tbl_income


,sa2,sa2_name,total_earners,median_age,income_sum,income_median,income_mean
0,101021007,Braidwood,2361,51,120763285,41593,51149
1,101021008,Karabar,5100,42,338308979,61777,66335
2,101021009,Queanbeyan,6697,39,441160946,60119,65874
3,101021010,Queanbeyan - East,3393,40,237035656,63051,69860
4,101021011,Queanbeyan Region,12821,44,1050285692,71420,81919
...,...,...,...,...,...,...,...
2290,801101137,Molonglo,np,np,np,np,np
2291,801101138,Molonglo - North,np,np,np,np,np
2292,801101139,Wright,2121,35,182421537,79150,86007
2293,801111140,ACT - South West,355,40,26069449,64227,73435


In [4]:
tbl_income[tbl_income["sa2"]==508041205]

,sa2,sa2_name,total_earners,median_age,income_sum,income_median,income_mean


In [5]:
# 一定要存吗？
# tbl_income.to_csv(curated_data+"clean_annual.csv")

<h3>1.2 Clean "population"</h3>

In [5]:
tbl_population.drop(tbl_population.columns[0:8], axis=1, inplace=True)
tbl_population.drop(tbl_population.columns[1:22], axis=1, inplace=True)
tbl_population.drop(tbl_population.columns[2:6], axis=1, inplace=True)
tbl_population

,SA2 code,2021,km2,persons/km2
0,101021007.0,4330.0,3418.4,1.3
1,101021008.0,8546.0,7.0,1223.9
2,101021009.0,11370.0,4.8,2387.7
3,101021010.0,5093.0,13.0,391.7
4,101021012.0,12743.0,13.7,931.9
...,...,...,...,...
2456,NaN,NaN,NaN,NaN
2457,NaN,NaN,NaN,NaN
2458,NaN,NaN,NaN,NaN
2459,NaN,NaN,NaN,NaN


In [6]:
tbl_population.columns = ["sa2", "2021_population", "km2", "persons/km2"]
tbl_population = tbl_population.dropna(axis=0, how='any', thresh=None, subset="sa2", inplace=False)
tbl_population['sa2'] = tbl_population['sa2'].astype(float).astype(int)
tbl_population

/var/folders/hg/_l53xsd126v9m8h8mg6914ph0000gn/T/ipykernel_28209/1461398505.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tbl_population['sa2'] = tbl_population['sa2'].astype(float).astype(int)


,sa2,2021_population,km2,persons/km2
0,101021007,4330.0,3418.4,1.3
1,101021008,8546.0,7.0,1223.9
2,101021009,11370.0,4.8,2387.7
3,101021010,5093.0,13.0,391.7
4,101021012,12743.0,13.7,931.9
...,...,...,...,...
2449,801111141,67.0,1202.8,0.1
2450,901011001,1716.0,136.1,12.6
2451,901021002,602.0,13.7,43.9
2452,901031003,310.0,67.2,4.6


In [8]:
# 要存吗？
# tbl_population.to_csv(curated_data+"clean_population.csv")

<h3>1.3 Clean "postcode_sa2"</h3>

In [7]:
postcode_2011_SA2_2011 = pd.ExcelFile(raw_data+"1270055006_CG_POSTCODE_2011_SA2_2011.xls")
postcode_sa2 = pd.read_excel(postcode_2011_SA2_2011, "Table 3")

In [8]:
postcode_sa2.drop(np.arange(0,6), inplace=True)
postcode_sa2.drop(np.arange(5994,5997), inplace=True)

postcode_sa2.drop(postcode_sa2.columns[3:6], axis=1, inplace=True)
postcode_sa2.drop(postcode_sa2.columns[0], axis=1, inplace=True)
postcode_sa2 = postcode_sa2.reset_index(drop=True)

postcode_sa2.columns = ['postcode', 'sa2']
postcode_sa2['sa2'] = pd.to_numeric(postcode_sa2['sa2'])

In [12]:
# postcode_sa2.to_csv(curated_data+"postcode_sa2.csv")

<h2>2. Join external data</h2>

<h3>2.1 Join "annual income" and "population"</h3>

In [10]:
merge_income_population = tbl_income.merge(tbl_population, on='sa2', how='inner')

In [11]:
merge_income_population

,sa2,sa2_name,total_earners,median_age,income_sum,income_median,income_mean,2021_population,km2,persons/km2
0,101021007,Braidwood,2361,51,120763285,41593,51149,4330.0,3418.4,1.3
1,101021008,Karabar,5100,42,338308979,61777,66335,8546.0,7.0,1223.9
2,101021009,Queanbeyan,6697,39,441160946,60119,65874,11370.0,4.8,2387.7
3,101021010,Queanbeyan - East,3393,40,237035656,63051,69860,5093.0,13.0,391.7
4,101021012,Queanbeyan West - Jerrabomberra,8476,44,725602722,73851,85607,12743.0,13.7,931.9
...,...,...,...,...,...,...,...,...,...,...
2142,801101136,Denman Prospect,416,38,41056005,90337,98692,2719.0,4.7,573.2
2143,801101137,Molonglo,np,np,np,np,np,0.0,2.2,0.0
2144,801101139,Wright,2121,35,182421537,79150,86007,3806.0,1.3,2993.1
2145,801111140,ACT - South West,355,40,26069449,64227,73435,554.0,416.8,1.3


<h3>2.2 Join "merge_income_population" and "postcode_sa2"</h3>

In [12]:
merge_income_population_postcode = merge_income_population.merge(postcode_sa2, on='sa2', how='left')
merge_income_population_postcode = merge_income_population_postcode.dropna(axis=0, how='any', thresh=None, subset="postcode", inplace=False)

In [13]:
merge_income_population_postcode = merge_income_population_postcode[merge_income_population_postcode["total_earners"] != "np"]

In [14]:
merge_income_population_postcode["total_earners"] = pd.to_numeric(merge_income_population_postcode["total_earners"])
merge_income_population_postcode["median_age"] = pd.to_numeric(merge_income_population_postcode["median_age"])
merge_income_population_postcode["income_sum"] = pd.to_numeric(merge_income_population_postcode["income_sum"])
merge_income_population_postcode["income_median"] = pd.to_numeric(merge_income_population_postcode["income_median"])
merge_income_population_postcode["income_mean"] = pd.to_numeric(merge_income_population_postcode["income_mean"])

In [15]:
agg_income_population_postcode = merge_income_population_postcode.groupby("postcode").agg(
    {
    "total_earners": "sum",
    "median_age": "mean",
    "income_sum": "sum",
    "2021_population": "sum",
    "km2": "sum"
    }
).reset_index()

In [16]:
agg_income_population_postcode

,postcode,total_earners,median_age,income_sum,2021_population,km2
0,0800,5632,33.000000,420609031,7679.0,3.2
1,0810,21932,39.583333,1574969237,36216.0,24.3
2,0812,11443,42.000000,801236575,19888.0,12.0
3,0820,15219,39.555556,1235996605,22877.0,67.8
4,0822,21869,40.142857,1425664118,61406.0,248437.3
...,...,...,...,...,...,...
2416,7466,1985,45.000000,115705864,4373.0,3931.6
2417,7467,1985,45.000000,115705864,4373.0,3931.6
2418,7468,1985,45.000000,115705864,4373.0,3931.6
2419,7469,1985,45.000000,115705864,4373.0,3931.6


<h3>2.3 Join "agg_income_population_postcode" and "mechant_consumer_info" (internal data)</h3>

In [63]:
merchant_consumer_info = pd.read_parquet(curated_data+"mechant_consumer_info")

In [65]:
merchant_consumer_info["consumer_postcode"] = merchant_consumer_info["consumer_postcode"].str.pad(width=4, fillchar="0", side="left")

In [66]:
final_table = agg_income_population_postcode.merge(merchant_consumer_info, left_on='postcode', right_on='consumer_postcode', how='right')
len(final_table[final_table["income_sum"].isnull()]["consumer_postcode"].unique())
# print(f"Mismatched (null) transactions dropped: {mechant_consumer_info.shape[0] - final_table.shape[0]}")

746

In [19]:
final_table

,postcode,total_earners,median_age,income_sum,2021_population,km2,merchant_name,revenue_level,products,take_rate,dollar_value,order_year,order_month,order_day,consumer,consumer_address,consumer_state,consumer_postcode,consumer_gender
0,2010,36510,35.666667,3692142068,45272.0,3.7,Magna Sed Institute,e,"[cable, satellite, and, other, pay, television...",0.31,24.661918,2021,7,5,Frederick Hughes,271 Cynthia Center,NSW,2010,Male
1,2010,36510,35.666667,3692142068,45272.0,3.7,Dignissim Lacus Corporation,d,"[gift, card, novelty, and, souvenir, shops]",0.59,150.987220,2021,7,9,Lynn Weeks PhD,8031 Katherine Street Apt. 065,NSW,2010,Female
2,2010,36510,35.666667,3692142068,45272.0,3.7,Aliquet Molestie Limited,d,"[stationery, office, supplies, and, printing, ...",1.19,1180.483980,2021,7,22,Alexandra Gomez,5717 Berger Route Suite 186,NSW,2010,Female
3,2010,36510,35.666667,3692142068,45272.0,3.7,Elit Etiam Laoreet Foundation,d,"[stationery, office, supplies, and, printing, ...",0.85,115.180265,2021,4,14,Lynn Weeks PhD,8031 Katherine Street Apt. 065,NSW,2010,Female
4,2010,36510,35.666667,3692142068,45272.0,3.7,Tempor Augue Limited,d,"[florists, supplies, nursery, stock, and, flow...",1.05,140.416636,2021,5,23,Frederick Hughes,271 Cynthia Center,NSW,2010,Male
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2589216,7470,1985,45.000000,115705864,4373.0,3931.6,Ipsum Primis Associates,a,"[watch, clock, and, jewelry, repair, shops]",6.94,53.097323,2021,4,24,Michael Bush,47924 Jessica Drive Apt. 344,TAS,7470,Male
2589217,7470,1985,45.000000,115705864,4373.0,3931.6,Ipsum Primis Associates,a,"[watch, clock, and, jewelry, repair, shops]",6.94,27.369566,2021,5,21,Shannon Petty,386 Aaron Manors,TAS,7470,Female
2589218,7470,1985,45.000000,115705864,4373.0,3931.6,Ipsum Primis Associates,a,"[watch, clock, and, jewelry, repair, shops]",6.94,56.702387,2021,7,17,Shannon Petty,386 Aaron Manors,TAS,7470,Female
2589219,7470,1985,45.000000,115705864,4373.0,3931.6,Ipsum Primis Associates,a,"[watch, clock, and, jewelry, repair, shops]",6.94,12.181018,2021,4,25,Shannon Petty,386 Aaron Manors,TAS,7470,Female


In [20]:
final_table.write.parquet("../data/curated/transact_abs")

AttributeError: 'DataFrame' object has no attribute 'write'